In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.
# !pip install wget
# !pip install git+https://github.com/NVIDIA/apex.git
# !pip install nemo_toolkit[nlp]
# !pip install unidecode
import os
import nemo
import nemo.collections.nlp as nemo_nlp
import numpy as np
import time
import errno
import json

from nemo.backends.pytorch.common.losses import CrossEntropyLossNM
from nemo.collections.nlp.nm.data_layers import BertQuestionAnsweringDataLayer
from nemo.collections.nlp.nm.trainables import TokenClassifier
from nemo.collections.nlp.callbacks.qa_squad_callback import eval_epochs_done_callback, eval_iter_callback
from nemo.utils.lr_policies import get_lr_policy
from nemo import logging

# Introduction
BioBERT has the same network architecture as the original BERT, but instead of Wikipedia and BookCorpus it is pretrained on PubMed, a large biomedical text corpus, which achieves better performance in biomedical downstream tasks, such as question answering(QA), named entity recognition(NER) and relationship extraction(RE). This model was trained for 1M steps. For more information please refer to the original paper https://academic.oup.com/bioinformatics/article/36/4/1234/5566506.  For details about BERT please refer to https://ngc.nvidia.com/catalog/models/nvidia:bertbaseuncasedfornemo.

BioMegatron is an in house model, using Megatron https://github.com/NVIDIA/Megatron-LM pretrained on PubMed. The accuracy is better than using BioBERT on downstream tasks


In this notebook we're going to showcase how to train BioBERT/BioMegatron on a biomedical question answering (QA) dataset.

# Download model  checkpoint
Download BioBert/BioMegatron checkpoints finetuned on SQuADv1.1 from  NGC: https://ngc.nvidia.com/catalog/models. Alternatively, you can also download BioBert/BioMegatron checkpoints and do the finetuning on SQuADv1.1 locally. This will take some time. For this, follow instructions at https://ngc.nvidia.com/catalog/models/nvidia:bertbaseuncasedsquadv1. 
    Then, put the encoder weights at `./checkpoints/biobert/qa_squad/BERT.pt` or `./checkpoints/biomegatron/qa_squad/BERT.pt`, the model head weights at `./checkpoints/biobert/qa_squad/TokenClassifier.pt` or `./checkpoints/biomegatron/qa_squad/TokenClassifier.pt` and the model configuration file at `./checkpoints/biobert/qa_squad/bert_config.json` or `./checkpoints/biomegatron/qa_squad/bert_config.json`.

In [ ]:
# Set which model to use.
model_type="biobert" # "biomegatron"
base_checkpoint_path={'biobert': './checkpoints/biobert/qa_squad', 'biomegatron': './checkpoints/biomegatron/qa_squad'}
pretrained_model_name={'biobert': 'bert-base-cased', 'biomegatron': 'megatron-bert-uncased'}
do_lower_case={'biobert': False, 'biomegatron': True}

In [ ]:
# the checkpoints are available from NGC: https://ngc.nvidia.com/catalog/models
CHECKPOINT_ENCODER = os.path.join(base_checkpoint_path[model_type], 'BERT.pt')
CHECKPOINT_HEAD = os.path.join(base_checkpoint_path[model_type], 'TokenClassifier.pt')
CHECKPOINT_CONFIG = os.path.join(base_checkpoint_path[model_type], 'bert_config.json')
    
if not os.path.exists(CHECKPOINT_ENCODER):
    raise OSError(errno.ENOENT, os.strerror(errno.ENOENT), CHECKPOINT_ENCODER)

if not os.path.exists(CHECKPOINT_HEAD):
    raise OSError(errno.ENOENT, os.strerror(errno.ENOENT), CHECKPOINT_HEAD)
    
if not os.path.exists(CHECKPOINT_CONFIG):
    raise OSError(errno.ENOENT, os.strerror(errno.ENOENT), CHECKPOINT_CONFIG)

# Download training data
In this example we download the QA dataset BioASQ6b to ./datasets/bioasq. Before using the files in this repository, you must first register BioASQ website and download the [BioASQ Task B](http://participants-area.bioasq.org/Tasks/A/getData/) data.

In [ ]:
data_dir="./datasets"
dataset="BioASQ"
if not os.path.exists(f"{data_dir}/{dataset}"):
    !python ../question_answering/get_bioasq.py --data_dir=$data_dir
!ls -l $data_dir/$dataset

After the previous step, you should have a ./datasets/BioASQ folder that contains the following files:

- 6B1_golden.json
- 6B2_golden.json
- 6B3_golden.json
- 6B4_golden.json
- 6B5_golden.json
- BioASQ-6b/train/Full-Abstract/BioASQ-train-factoid-6b-full-annotated.json
- BioASQ-6b/test/BioASQ-6b/test/Full-Abstract/BioASQ-test-factoid-6b-1.json
- BioASQ-6b/test/BioASQ-6b/test/Full-Abstract/BioASQ-test-factoid-6b-2.json
- BioASQ-6b/test/BioASQ-6b/test/Full-Abstract/BioASQ-test-factoid-6b-3.json
- BioASQ-6b/test/BioASQ-6b/test/Full-Abstract/BioASQ-test-factoid-6b-4.json
- BioASQ-6b/test/BioASQ-6b/test/Full-Abstract/BioASQ-test-factoid-6b-5.json

The format of the data described in NeMo docs.

# Create Neural Modules

In [ ]:
model_checkpoint=CHECKPOINT_ENCODER # language model encoder file
head_checkpoint=CHECKPOINT_HEAD # language model encoder file
model_config=CHECKPOINT_CONFIG # model configuration file
work_dir="output_bioasq"
train_file=f"{data_dir}/{dataset}/BioASQ-6b/train/Full-Abstract/BioASQ-train-factoid-6b-full-annotated.json"
doc_stride=128
max_query_length=64
max_seq_length=384
batch_size=12
version_2_with_negative=False

nf = nemo.core.NeuralModuleFactory(
    backend=nemo.core.Backend.PyTorch,
    placement=nemo.core.DeviceType.GPU,
    log_dir=work_dir
)
model = nemo_nlp.nm.trainables.get_pretrained_lm_model(
        config=model_config, pretrained_model_name=pretrained_model_name[model_type], checkpoint=model_checkpoint
    )
tokenizer = nemo.collections.nlp.data.tokenizers.get_tokenizer(
    tokenizer_name='nemobert',
    pretrained_model_name=pretrained_model_name[model_type],
    do_lower_case=do_lower_case[model_type]
)
hidden_size = model.hidden_size
qa_head = TokenClassifier(
    hidden_size=hidden_size, num_classes=2, num_layers=1, log_softmax=False, name="TokenClassifier"
)
qa_head.restore_from(head_checkpoint)
task_loss = nemo_nlp.nm.losses.SpanningLoss()
# create training data layer, preprocessing takes a while. If you want to cache preprocessed data for future reuse use --use_cache=True
train_data_layer = BertQuestionAnsweringDataLayer(
    mode="train",
    tokenizer=tokenizer,
    version_2_with_negative=version_2_with_negative,
    data_file=train_file,
    max_query_length=max_query_length,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    batch_size=batch_size,
    shuffle=True,
    use_cache=True
)

# Creating Neural graph

In [ ]:
train_data = train_data_layer()
hidden_states = model(input_ids=train_data.input_ids, token_type_ids=train_data.input_type_ids, attention_mask=train_data.input_mask)
qa_output = qa_head(hidden_states=hidden_states)
loss = task_loss(logits=qa_output, start_positions=train_data.start_positions, end_positions=train_data.end_positions)
# If you're training on multiple GPUs, this should be
# len(train_data_layer) // (batch_size * batches_per_step * num_gpus)
train_steps_per_epoch = len(train_data_layer) // batch_size
logging.info(f"doing {train_steps_per_epoch} steps per epoch")

# Create Callbacks


In [ ]:
train_callback = nemo.core.SimpleLossLoggerCallback(
    tensors=[loss.loss],
    print_func=lambda x: logging.info("Loss: {:.3f}".format(x[0].item())),
    get_tb_values=lambda x: [["loss", x[0]]],
    step_freq=100,
    tb_writer=nf.tb_writer,
)
ckpt_callback = nemo.core.CheckpointCallback(
    folder=nf.checkpoint_dir, epoch_freq=1, step_freq=-1
)

# Training
this may take more than an hour.

In [ ]:
num_epochs=5
lr=5e-6
lr_warmup_proportion=0
weight_decay=0
lr_policy_fn = get_lr_policy("WarmupAnnealing", total_steps=num_epochs * train_steps_per_epoch, warmup_ratio=lr_warmup_proportion
)
nf.reset_trainer()
nf.train(
    tensors_to_optimize=[loss.loss],
    callbacks=[train_callback, ckpt_callback],
    lr_policy=lr_policy_fn,
    optimizer="adam_w",
    optimization_params={"num_epochs": num_epochs, "lr": lr, "weight_decay": weight_decay},
)

# Inference
Do inference on test data e.g. 6b-1

In [ ]:
test_dataset="6b"
test_dataset_idx="1"

In [ ]:
test_file=f"{data_dir}/{dataset}/BioASQ-6b/test/Full-Abstract/BioASQ-test-factoid-{test_dataset}-{test_dataset_idx}.json"
logging.info(f"using test file {test_file}")
test_data_layer = BertQuestionAnsweringDataLayer(
    mode="test",
    tokenizer=tokenizer,
    version_2_with_negative=version_2_with_negative,
    data_file=test_file,
    max_query_length=max_query_length,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    batch_size=1,
    shuffle=False,
    use_cache=False
)

# Creating Neural test graph
test_data = test_data_layer()
test_hidden_states = model(input_ids=test_data.input_ids, token_type_ids=test_data.input_type_ids, attention_mask=test_data.input_mask)
test_qa_output = qa_head(hidden_states=test_hidden_states)
test_tensors=[test_data.unique_ids, test_qa_output]

In [ ]:
n_best_size=20
null_score_diff_threshold=0
max_answer_length=30
logging.info(f"work dir {work_dir}, checkpoint dir {nf.checkpoint_dir}")
output_prediction_file=f"{work_dir}/predictions.json"
output_nbest_file=f"{work_dir}/nbest.json"
evaluated_tensors = nf.infer(
    tensors=test_tensors, cache=False, offload_to_cpu=False, checkpoint_dir=nf.checkpoint_dir
)
unique_ids = []
for t in evaluated_tensors[0]:
    unique_ids.extend(t.tolist())
logits = []
for t in evaluated_tensors[1]:
    logits.extend(t.tolist())
start_logits, end_logits = np.split(np.asarray(logits), 2, axis=-1)
(all_predictions, all_nbest, scores_diff) = test_data_layer.dataset.get_predictions(
    unique_ids=unique_ids,
    start_logits=start_logits,
    end_logits=end_logits,
    n_best_size=n_best_size,
    max_answer_length=max_answer_length,
    version_2_with_negative=version_2_with_negative,
    null_score_diff_threshold=null_score_diff_threshold,
    do_lower_case=do_lower_case[model_type],
)
with open(output_nbest_file, "w") as writer:
    writer.write(json.dumps(all_nbest, indent=4) + "\n")
with open(output_prediction_file, "w") as writer:
    writer.write(json.dumps(all_predictions, indent=4) + "\n")


In [34]:
# a test question example would be 
!head -n 15 $test_file

{
  "version": "BioASQ6b", 
  "data": [
    {
      "title": "BioASQ6b", 
      "paragraphs": [
        {
          "context": "emMAW: computing minimal absent words in external memory. Motivation: The biological significance of minimal absent words has been investigated in genomes of organisms from all domains of life. For instance, three minimal absent words of the human genome were found in Ebola virus genomes. There exists an O(n) -time and O(n) -space algorithm for computing all minimal absent words of a sequence of length n on a fixed-sized alphabet based on suffix arrays. A standard implementation of this algorithm, when applied to a large sequence of length n , requires more than 20 n \u00a0bytes of RAM. Such memory requirements are a significant hurdle to the computation of minimal absent words in large datasets. Results: We present emMAW, the first external-memory algorithm for computing minimal absent words. A free open-source implementation of our algorithm is made availabl

In [35]:
# the corresponding first 2 best answers of the n-best list prediction with probabilities. The most likely answer is "emMAW" with a probability of 0.995
!head -n 20 $output_nbest_file

{
    "5a6a3335b750ff4455000025_000": [
        {
            "text": "emMAW",
            "probability": 0.9950947775946923,
            "start_logit": [
                5.9411702156066895
            ],
            "end_logit": [
                5.900936126708984
            ]
        },
        {
            "text": "emMAW,",
            "probability": 0.001918466002056132,
            "start_logit": [
                5.9411702156066895
            ],
            "end_logit": [
                -0.3503759503364563


In [36]:
# the golden label can be found in this following file under "exact_answer". In this case, the it is equal to the prediction which is "emMAW"
prefix=test_dataset.upper()
suffix=f"{test_dataset_idx}_golden.json"
!grep -B 7 "5a6a3335b750ff4455000025" $data_dir/$dataset/$prefix$suffix

      "exact_answer": [
        [
          "emMAW"
        ]
      ], 
      "concepts": [], 
      "type": "factoid", 
      "id": "5a6a3335b750ff4455000025", 


# Evaluate inference output with BioASQ metrics

In [ ]:
if not os.path.exists('bioasq-biobert'):
    print("clone https://github.com/dmis-lab/bioasq-biobert.git")
    !git clone https://github.com/dmis-lab/bioasq-biobert.git && cd bioasq-biobert && git fetch origin pull/12/head:fix_indentation && git checkout fix_indentation && cd ..
if not os.path.exists('Evaluation-Measures'):
    print("clone https://github.com/BioASQ/Evaluation-Measures.git")
    !git clone https://github.com/BioASQ/Evaluation-Measures.git && git checkout cd93f3b8eb290c965d18ef466ee28a0bcf451e5d

In [ ]:
transformed_nbest_dir=f"{work_dir}/transformed_nbest"
!mkdir -p $transformed_nbest_dir
!python bioasq-biobert/biocodes/transform_n2b_factoid.py --nbest_path=$output_nbest_file --output_path=$transformed_nbest_dir

In [ ]:
prefix=test_dataset.upper()
suffix=f"{test_dataset_idx}_golden.json"
! java -Xmx10G -cp Evaluation-Measures/flat/BioASQEvaluation/dist/BioASQEvaluation.jar evaluation.EvaluatorTask1b -phaseB -e 5 $data_dir/$dataset/$prefix$suffix $transformed_nbest_dir/BioASQform_BioASQ-answer.json

The result will look something like this for BioBERT:

```0.0 0.45161290322580644 0.6774193548387096 0.5403225806451613 0.0 0.0 0.0 0.0 0.0 0.0```

where the second, third and fourth numbers will be strict accuracy (SAcc), lenient accuracy (LAcc) and mean reciprocal rank (MRR) for factoid
questions respectively.